In [1]:
import pandas as pd
import numpy as np
import pickle
import numpy as np
from causalinference import CausalModel

import xgboost
print("xgboost", xgboost.__version__)

import sqlite3

xgboost 1.3.3


In [5]:
with open('../data/program_zip_dates.p', 'rb') as fp:
    prog_map = pickle.load(fp)

In [2]:
conn_proj = sqlite3.connect("../data/Berkeley.db")
cursor_proj = conn_proj.cursor()

query_frame = """
SELECT *

FROM installation

"""


df = pd.read_sql_query(query_frame, conn_proj)


conn_proj.close

<function Connection.close>

In [3]:
keep_cols = ['size', 'price','discount']
keep_cols_2 = ['p_type_31','p_type_32', 'p_type_33', 'p_type_37',
             'p_type_46', 'p_type_50', 'p_type_78', 'p_type_87', 'p_type_88', 'p_type_92', 'p_type_86']

g_2 = df.groupby(['month','zipcode'])[keep_cols].mean().reset_index()

df['size'] = df['size'].astype('bool')
grouped_df = df.groupby(['month','zipcode'])['size'].sum().reset_index().rename(
    columns = {'size':'i_count'})


grouped_df = grouped_df.merge(g_2, left_on=['zipcode','month'], right_on=['zipcode','month'])

grouped_df.head()

,month,zipcode,i_count,size,price,discount
0,3.0,00501,0.0,0.0,11644.84,0.0
1,3.0,00831,0.0,0.0,11644.84,0.0
2,3.0,01001,0.0,0.0,11644.84,0.0
3,3.0,01002,0.0,0.0,11644.84,0.0
4,3.0,01004,0.0,0.0,11644.84,0.0


In [9]:
trial_df = grouped_df.copy()

for program in [87,88]:
    for num_mo in np.linspace(-9,9,19):
        def in_effect(x):

            z = x[0]
            d = x[1]

            p_type = program

            output = -1

            try:
                items = []
                for item in prog_map[p_type][z]:
                    items.append(item[1])
                    if output != 1:
                        if item[1] <= d <= item[1] + num_mo:
                            output = 1
                        elif item[1] - num_mo <= d <= item[1]:
                            output = 0
                        #elif d == item[1]:
                        #    output = .5
                #if d < min(items):
                #    output = 0


            except:
                output = -1

            return output

        trial_df['p_type_' + str(program)] = [in_effect(x) for x in zip(trial_df.zipcode,trial_df.month)]
        corr_df = trial_df[['discount','p_type_' + str(program)]]
        corr_df = corr_df[corr_df['p_type_' + str(program)] >= 0]
        corr = corr_df['p_type_' + str(program)].corr(corr_df.discount)
        print('p_type ' + str(program) + ' correlation for ' + str(num_mo) + ' months is ' + str(corr))

p_type 87 correlation for -9.0 months is nan
p_type 87 correlation for -8.0 months is nan
p_type 87 correlation for -7.0 months is nan
p_type 87 correlation for -6.0 months is nan
p_type 87 correlation for -5.0 months is nan
p_type 87 correlation for -4.0 months is nan
p_type 87 correlation for -3.0 months is nan
p_type 87 correlation for -2.0 months is nan
p_type 87 correlation for -1.0 months is nan
p_type 87 correlation for 0.0 months is nan
p_type 87 correlation for 1.0 months is 0.033361370035194834
p_type 87 correlation for 2.0 months is 0.023603180707108772
p_type 87 correlation for 3.0 months is 0.011907651205182257
p_type 87 correlation for 4.0 months is 0.010780145402491103
p_type 87 correlation for 5.0 months is 0.00715989334125539
p_type 87 correlation for 6.0 months is 0.0003942424352317307
p_type 87 correlation for 7.0 months is -0.0036383705170894604
p_type 87 correlation for 8.0 months is -0.003332278715692047
p_type 87 correlation for 9.0 months is -0.00189734424522011

In [4]:
np.linspace(1,18,18)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18.])